# IBM Quantum Experience Lab - Lab script

Please use this notebook as a template for the work that you will submit for assessment for this lab exercise.

Student Names: *Sara Cender, Eleanor Kneip, Stasiu Wolanski*

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator, Pauli, Statevector, SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
import numpy as np

%set_env QXToken=656a2556246117cd382fba4bb957892cece054dad6a3102c567c2ef3ccd03b436e78e3c0529582e760cce684ff1c2c60b2e25ac01bd320e043db27d995a09f6f




env: QXToken=656a2556246117cd382fba4bb957892cece054dad6a3102c567c2ef3ccd03b436e78e3c0529582e760cce684ff1c2c60b2e25ac01bd320e043db27d995a09f6f


## Warm up exercise

In [ ]:
def z_expected_value(qc: QuantumCircuit, true_expected_value: float, real_device = False):
    """Calculate expectation of Z, using Aer simulator."""
    if not real_device:
        shots = 1_000_000
        backend = AerSimulator()
        job = backend.run(qc, shots=shots)
        result = job.result()
        counts = result.get_counts()

    else:
        # Initialize runtime service
        service = QiskitRuntimeService(channel="ibm_quantum", token = '656a2556246117cd382fba4bb957892cece054dad6a3102c567c2ef3ccd03b436e78e3c0529582e760cce684ff1c2c60b2e25ac01bd320e043db27d995a09f6f')
        # Choose backend (example: least busy backend with 1+ qubit)
        backend = service.least_busy(operational=True, min_num_qubits=1)

        sampler = Sampler(mode=backend)
        job = sampler.run([qc], shots=1000)
        result = job.result()
        counts = result.quasi_dists[0]

    # Calculate expectation value of Z
    expectation_Z = (counts.get('0', 0) - counts.get('1', 0)) / shots
    print(f"Expectation value of Z: {expectation_Z}")

    print(f"expected error is {2*np.sqrt(counts.get('0', 0) * counts.get('1', 0) / shots**3):.1e}")
    print(f"actual error is {np.abs(expectation_Z - true_expected_value):.1e}")
    

In [59]:
qr = QuantumRegister(1)
cr = ClassicalRegister(1)
qc = QuantumCircuit(qr, cr)

qc.h([0])
qc.measure([0], [0])

z_expected_value(qc, 0, real_device=True)

ValueError: An invalid Sampler pub-like was given (<class 'qiskit._accelerate.circuit.CircuitInstruction'>). If you want to run a single circuit, you need to wrap it with `[]` like `sampler.run([circuit])` instead of `sampler.run(circuit)`.

## Method I

In [ ]:
def run_circuit_method_1(phi, theta, true_expected_value):
    """Calculate expectation of M using Method I."""
    psi = Statevector([np.cos(phi), 1j * np.sin(phi)])

    U = UnitaryGate((1/np.sqrt(2))*np.array([[1,1],[np.exp(1j*theta), -np.exp(1j*theta)]]).T.conj())

    qr = QuantumRegister(1)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr, cr)

    qc.initialize(psi, 0)

    qc.append(U, qr)

    qc.measure(0, 0)

    qc.draw()

    z_expected_value(qc, true_expected_value)

def true_answer(theta, phi):
    """Calculate true expected value of M."""
    psi = np.array([np.cos(phi), 1j * np.sin(phi)])
    M = np.array([[0, np.exp(-theta*1j)],[np.exp(theta*1j),0]])
    return psi.conj() @ M @ psi.T

In [ ]:
# Pair I

phi = np.pi/2
theta = np.pi/4
true_expected_value = true_answer(theta, phi)
print(f"{true_expected_value=}")

run_circuit_method_1(phi, theta, true_expected_value)

true_expected_value=np.complex128(8.659560562354932e-17+0j)
Expectation value of Z: -0.000644
expected error is 1.0e-03
actual error is 6.4e-04


In [ ]:
# Pair II

phi = np.pi/4
theta = np.pi/8
true_expected_value = true_answer(theta, phi)
print(f"{true_expected_value=}")

run_circuit_method_1(phi, theta, true_expected_value)

true_expected_value=np.complex128(0.3826834323650897+0j)
Expectation value of Z: 0.382356
expected error is 9.2e-04
actual error is 3.3e-04


# Method II

In [ ]:
def measure_X_and_Y_exps(qc: QuantumCircuit):
    """Measure a circuit with respect to X and Y observables."""
    observables = [SparsePauliOp('X'),SparsePauliOp('Y')]

    estimator = Estimator(AerSimulator())
    pub = (qc, observables)

    job = estimator.run(pubs=[pub])
    print(job.result())
    return job.result()[0].data.evs

def run_circuit_method_2(phi, theta):
    """Calculate expectation of M using Method II."""
    psi = Statevector([np.cos(phi), 1j * np.sin(phi)])

    qr = QuantumRegister(1)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr, cr)

    qc.initialize(psi, 0)

    x_exp, y_exp = measure_X_and_Y_exps(qc)

    m_exp = np.cos(theta)* x_exp + np.sin(theta)*y_exp
    print(f"Expected value of m: {m_exp}")
    

In [ ]:
# Pair I

phi = np.pi/2
theta = np.pi/4
true_expected_value = true_answer(theta, phi)
print(f"{true_expected_value=}")

run_circuit_method_2(phi, theta)

true_expected_value=np.complex128(8.659560562354932e-17+0j)
PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(2,), dtype=float64>), stds=np.ndarray(<shape=(2,), dtype=float64>), shape=(2,)), metadata={'target_precision': 0.015625, 'shots': 4096, 'circuit_metadata': {}})], metadata={'version': 2})
Expected value of m: 0.0017263349150062213


In [ ]:
# Pair II

phi = np.pi/4
theta = np.pi/8
true_expected_value = true_answer(theta, phi)
print(f"{true_expected_value=}")

run_circuit_method_2(phi, theta)

true_expected_value=np.complex128(0.3826834323650897+0j)
PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(2,), dtype=float64>), stds=np.ndarray(<shape=(2,), dtype=float64>), shape=(2,)), metadata={'target_precision': 0.015625, 'shots': 4096, 'circuit_metadata': {}})], metadata={'version': 2})
Expected value of m: 0.39260791953073837
